# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.12/lib/netstandard2.1/NetMQ.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        // use runtime = new NetMQ.NetMQRuntime ()
        use request = new NetMQ.Sockets.RequestSocket ()
        request.Connect $"tcp://127.0.0.1:{port}"

        let msg = NetMQ.NetMQMessage ()
        msg.Append json
        
        NetMQ.OutgoingSocketExtensions.SendMultipartMessage (request, msg)
        let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
        // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
        let result = result |> Seq.toList |> List.map (fun x -> x.ConvertToString ())
        trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result: {result}") getLocals
        return true
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return false
}

In [ ]:
let inline sendObj port obj =
    let json = System.Text.Json.JsonSerializer.Serialize obj
    sendJson port json

In [ ]:
let inline compile code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let inline writeCode fileName code =
        let filePath = tempDir </> fileName
        System.IO.File.WriteAllText (filePath, code)

    code
    |> writeCode "main.spi"

    "packages:\n    |core-\nmodules:\n    main"
    |> writeCode "package.spiproj"

    let stream, disposable = FileSystem.watchDirectory true tempDir

    let e = Event<_> ()

    let! _ =
        stream
        |> FSharp.Control.AsyncSeq.iterAsyncParallel (fun event -> async {
            // trace Debug (fun () -> $"event: {event}") getLocals
            match event with
            | _ticks, FileSystemChange.Changed ("main.fsx", Some content) ->
                e.Trigger content
                disposable.Dispose ()
            | _ -> ()
        })
        |> Async.StartChild

    let port = 13805

    let! _ =
        async {
            let! availablePort = Networking.getAvailablePort (Some 60) port
            if availablePort = port then
                let compilerPath =
                    "../../deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                    |> Path.GetFullPath

                let dllPath = compilerPath </> "Spiral.dll"
                // let commandsPath = compilerPath </> "compiler/supervisor/commands"

                let! exitCode, result = Runtime.executeAsync $@"dotnet ""{dllPath}"" port={port}"
                trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
        }
        |> Async.StartChild

    let inline getFileUri path =
        $"file:///{path}"

    let mainPath = tempDir </> "main.spi"

    let! mainText = mainPath |> System.IO.File.ReadAllTextAsync |> Async.AwaitTask

    let fileOpenObj = {| FileOpen = {| uri = mainPath |> getFileUri; spiText = mainText |} |}
    let! fileOpenResult = fileOpenObj |> sendObj port

    let buildFileObj = {| BuildFile = {| uri = mainPath |> getFileUri; backend = "Fsharp" |} |}
    let! buildFileResult = buildFileObj |> sendObj port

    let! content = Async.AwaitEvent e.Publish
    return content |> String.replace "\r\n" "\n"
}

In [ ]:
//// test


"""inl app () =
    0i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> compile
|> Async.runWithTimeout 6000
|> _equal (Some """let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
""")

02:13:17 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0213-1706-0600-0e1b619a3e15 / result: { CreationTime = 2023-07-29 2:13:17 AM
  Exists = true }
02:13:17 #2 [Debug] runWithTimeoutAsync / timeout: 60
02:13:17 #3 [Debug] executeAsync / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805
02:13:17 #4 [Debug]  34612: Server bound to: tcp://*:13805 & tcp://*:13806 / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805
02:13:17 #5 [Debug]  34612: pwd: C:\home\git\polyglot\apps\spiral / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=138

In [ ]:
//// test


"""inl write_line ~obj : () =
    $"System.Console.WriteLine !obj"

inl app () =
    write_line "error"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> compile
|> Async.runWithTimeout 6000
|> _equal (Some """let rec closure0 () () : int32 =
    let v0 : string = "error"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""")

02:13:21 #12 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230729-0213-2127-2712-2be3874f99d5 / result: { CreationTime = 2023-07-29 2:13:21 AM
  Exists = true }
02:13:21 #13 [Debug] runWithTimeoutAsync / timeout: 60
02:13:21 #14 [Debug] executeAsync / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805
02:13:21 #15 [Debug]  25808: Server bound to: tcp://*:13805 & tcp://*:13806 / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805
02:13:21 #16 [Debug]  25808: pwd: C:\home\git\polyglot\apps\spiral / workingDirectory: . / fileName: dotnet / arguments: "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" por